In [ ]:
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import transforms
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import sys
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weigths['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

In [ ]:
def get_files(path):
    files=list()
    for filepath,dirnames,filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(filepath,filename))
    return files
image_path="./data/image/infant_image.jpg"
save_path='./data/results_npy/'

vidname = os.path.basename(image_path)
vname = vidname.split('.')[0]
image = cv2.imread(image_path)
image = letterbox(image, 1024, stride=64, auto=True)[0]
image_ = image.copy()
image = transforms.ToTensor()(image)
image = torch.tensor(np.array([image.numpy()]))
if torch.cuda.is_available():
    image = image.half().to(device)   
output, _ = model(image)

output = non_max_suppression_kpt(output, 0.5, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)[0]
assert output.shape[0]>=1, f'{image_path}'
total_conf=output[:,8::3].sum(1,keepdim=True)
_, j = total_conf.max(0, keepdim=True)
output=output[int(j),:]
output=output.unsqueeze(0)

with torch.no_grad():
    output = output_to_keypoint(output)
assert output.shape[0]==1 and output.shape[1]==57, f'{image_path}'
np.savetxt(save_path + vname + '.txt', output,fmt='%f',delimiter=' ')